In [1]:
import numpy as np
import pandas as pd

In [51]:
amazon_review = pd.read_csv('amazon_baby.csv')
amazon_review.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [16]:
amazon_review['review'] = amazon_review['review'].astype(str)

In [17]:
def remove_punctuation(review):
    import string
    translator = str.maketrans('','',string.punctuation)
    return review.translate(translator)

In [18]:
amazon_review['clean_review'] = amazon_review['review'].apply(remove_punctuation)

In [20]:
amazon_review['clean_review'].head()

0    These flannel wipes are OK but in my opinion n...
1    it came early and was not disappointed i love ...
2    Very soft and comfortable and warmer than it l...
3    This is a product well worth the purchase  I h...
4    All of my kids have cried nonstop when I tried...
Name: clean_review, dtype: object

In [142]:
amazon_review = amazon_review.fillna({'review':'','clean_review':''})


In [24]:
amazon_review_class = amazon_review[amazon_review['rating']!=3]

In [33]:
amazon_review_class['sentiment'] = amazon_review_class['rating'].apply(lambda x:1 if x>3 else -1)

C:\SowmyaSoftwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
amazon_review_class.head(1)

,name,review,rating,clean_review,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1


In [47]:
import json
train_file = open('module-2-assignment-train-idx.json','r')
train_id = json.loads(train_file.read())
test_file = open('module-2-assignment-test-idx.json','r')
test_id = json.loads(test_file.read())

In [48]:
train_data = amazon_review_class.iloc[train_id]
test_data = amazon_review_class.iloc[test_id]

In [53]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [144]:
test_data.iloc[20743]

name            Fisher-Price Cradle 'N Swing,  My Little Snuga...
review          My husband and I cannot state enough how much ...
rating                                                          5
clean_review    My husband and I cannot state enough how much ...
sentiment                                                       1
Name: 114796, dtype: object

In [54]:
vectorizer  = CountVectorizer(token_pattern= r'\b\w+\b')

In [55]:
train_matrix = vectorizer.fit_transform(train_data['clean_review'])

In [56]:
test_matrix = vectorizer.transform(test_data['clean_review'])

In [57]:
train_matrix

<133416x121713 sparse matrix of type '<class 'numpy.int64'>'
	with 7327230 stored elements in Compressed Sparse Row format>

In [58]:
from sklearn.linear_model import LogisticRegression

In [87]:
sentiment_model = LogisticRegression()

In [88]:
sentiment_model.fit(train_matrix,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [89]:
sentiment_model.intercept_

array([1.3299993])

In [304]:
np.count_nonzero(sentiment_model.coef_>=0)# + np.count_nonzero(sentiment_model.coef_<0)

86890

In [91]:
sample_test_data = test_data[10:13]

In [92]:
sample_test_matrix = vectorizer.transform(sample_test_data['clean_review'])

In [97]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [98]:
sample_test_matrix

<3x121713 sparse matrix of type '<class 'numpy.int64'>'
	with 105 stored elements in Compressed Sparse Row format>

In [99]:
sample_test_data

,name,review,rating,clean_review,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [102]:
scores = sentiment_model.decision_function(sample_test_matrix)
scores

array([  5.58805454,  -3.18262063, -10.43030225])

In [104]:
1/(1+np.exp(-scores))

array([9.96271651e-01, 3.98250027e-02, 2.95232704e-05])

In [121]:
prob = pd.DataFrame(sentiment_model.predict_proba(test_matrix))


In [151]:
pos_index = prob.sort_values(by=1,ascending=False).iloc[0:20].index


In [152]:
test_data.iloc[pos_index]

,name,review,rating,clean_review,sentiment
114796,"Fisher-Price Cradle 'N Swing, My Little Snuga...",My husband and I cannot state enough how much ...,5,My husband and I cannot state enough how much ...,1
137034,Graco Pack 'n Play Element Playard - Flint,My husband and I assembled this Pack n' Play l...,4,My husband and I assembled this Pack n Play la...,1
147949,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,1
165593,Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatwa...,For the price this set is unbelievable- and tr...,5,For the price this set is unbelievable and tru...,1
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1
22586,"Britax Decathlon Convertible Car Seat, Tiffany",I researched a few different seats to put in o...,4,I researched a few different seats to put in o...,1
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1
87017,Baby Einstein Around The World Discovery Center,I am so HAPPY I brought this item for my 7 mon...,5,I am so HAPPY I brought this item for my 7 mon...,1
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1


In [158]:
neg_index = prob.sort_values(by=1).iloc[0:20].index

In [296]:
#test_data.iloc[neg_index]

In [160]:
prediction = sentiment_model.predict(test_matrix)

In [168]:
np.count_nonzero(prediction == test_data['sentiment'])/len(test_data)

0.9324154067674586

In [169]:
train_pred = sentiment_model.predict(train_matrix)

In [170]:
np.count_nonzero(train_pred == train_data['sentiment'])/len(train_data)

0.9686244528392397

In [171]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [172]:
vectorizer_sig = CountVectorizer(vocabulary=significant_words)

In [175]:
train_matrix_word_subset = vectorizer_sig.fit_transform(train_data['clean_review'])

In [176]:
test_matrix_word_subset = vectorizer_sig.transform(test_data['clean_review'])

In [177]:
simple_model = LogisticRegression()

In [179]:
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [180]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

In [275]:
simple_model_coeff = pd.DataFrame(simple_model.coef_.T,significant_words)
simple_model_coeff.head(1)

,0
love,1.36369


In [224]:
feature_list = vectorizer.get_feature_names()
type(feature_list)

list

In [253]:
index = [ind for ind,feat in enumerate(feature_list) if feat in significant_words]
feat = [feat for ind,feat in enumerate(feature_list) if feat in significant_words]

In [266]:
co_eff = sentiment_model.coef_.reshape(-1,1)[index]
co_eff.shape

(20, 1)

In [277]:
sentiment_model_coeff = pd.DataFrame(co_eff,feat)

In [278]:
sentiment_model_coeff.head(1)

,0
able,0.392691


In [284]:
pd.merge(sentiment_model_coeff,simple_model_coeff,left_index=True,right_index=True)

,0_x,0_y
able,0.392691,0.190909
broke,-1.389070,-1.651576
car,0.126216,0.058855
disappointed,-2.191301,-2.348298
easy,1.364508,1.192538
even,-0.465434,-0.511380
great,1.240148,0.944000
less,-0.275679,-0.209563
little,0.642607,0.520186
love,1.589991,1.363690


In [287]:
simple_train_prediction = simple_model.predict(train_matrix_word_subset)
simple_test_prediction = simple_model.predict(test_matrix_word_subset)

In [289]:
sentiment_model_train_prediction = sentiment_model.predict(train_matrix)
sentiment_model_test_prediction = sentiment_model.predict(test_matrix)

In [290]:
simple_train_accuracy = np.count_nonzero(simple_train_prediction == train_data['sentiment'])/len(train_data)
simple_test_accuracy = np.count_nonzero(simple_test_prediction == test_data['sentiment'])/len(test_data)
sentiment_train_accuracy = np.count_nonzero(sentiment_model_train_prediction == train_data['sentiment'])/len(train_data)
sentiment_test_accuracy = np.count_nonzero(sentiment_model_test_prediction == test_data['sentiment'])/len(test_data)

In [291]:
print(simple_train_accuracy)
print(sentiment_train_accuracy)

0.8668225700065959
0.9686244528392397


In [301]:
print(simple_test_accuracy)
print(sentiment_test_accuracy)

0.8693604511639069
0.9324154067674586


In [303]:
np.count_nonzero(train_data['sentiment'] ==1)/len(train_data) # majority classifier accuracy

0.8407087605684476